<img src='otus.png'>

# Анализ текстовых данных

### Какие задачи можно решать, обрабатывая текст?
"Мама мыла раму, и теперь она блестит"  
"Мама мыла раму, и теперь она сильно устала"  

"Кубок не помещался в чемодан, потому что он был слишком велик. Что именно было слишком велико, чемодан или кубок?"

http://commonsensereasoning.org/winograd.html


1. синтаксические задачи
  * разметка по частям речи и по морфологическим признакам
  * деление слов в тексте на морфемы (суффикс, приставка и пр.)
  * стемминг, лемматизация (?)
  * деление на предложения (инициалы и сокращения) и слова (китайский язык)
  * поиск имен и названий в тексте - сущностей
  * разрешение смысла слов в заданном контексте (замок)
  * построить синтаксическое дерево
  * определение того, к каким другим объектам относится слово
2. задачи на понимание текста, в которых есть "учитель"
  * предсказание следующего символа
  * информационный поиск
  * анализ тональности
  * выделение отношений и фактов
  * ответы на вопросы
3. понимание и порождение текста (оценка качества?)
  * порождение текста
  * автоматическое реферирование
  * машинный перевод
  * диалоговые модели (чат-бот)
  
Косвенные задачи:
  * описание изображения
  * распознавание речи
  
**Задачи бизнеса**:
  * распознавание речи (помощник)
  * чат-бот (замена техподдержки в решении большинства вопросов)
  * поиск точного ответа на вопрос в базе документов (например, база стандартов)
  * оценка мнения в социальных сетях о продукте
  * ... (ваши варианты?)

In [3]:
import nltk
# nltk.download()  # download lots of data

# От текста к простым моделям

## Разбиение на токены
**Def.**  
разбиение последовательности символов на части (токены), возможно, исключая из рассмотрения некоторые символы  
Наивный подход: разделить строку пробелами и выкинуть знаки препинания  


*Трисия любила Нью-Йорк, поскольку любовь к Нью-Йорку могла положительно повлиять на ее карьеру.*  


**Проблемы:**  
* my.email@mail.ru, 127.0.0.1
* С++, C#
* York University vs New York University
* Зависимость от языка (“Lebensversicherungsgesellschaftsangestellter”, “l’amour”)
Альтернатива: n-граммы

http://www.nltk.org/

In [4]:
from nltk.tokenize import RegexpTokenizer
tokenizer = RegexpTokenizer('\w+|[^\w\s]+')
s = u'Трисия любила Нью-Йорк, поскольку любовь к Нью-Йорку могла положительно повлиять на ее карьеру.'
for t in tokenizer.tokenize(s)[:7]: 
    print(t + " ::")

Трисия ::
любила ::
Нью ::
- ::
Йорк ::
, ::
поскольку ::


In [5]:
from ftfy import fix_text
print(fix_text(u'\001\033[36;44mI&#x92;m blue, da ba dee da ba doo&#133;\033[0m', normalization='NFKC'))

I'm blue, da ba dee da ba doo...


## Стоп-слова
**Def.**  
Наиболее частые слова в языке, не содержащие никакой информации о содержании текста



In [7]:
from nltk.corpus import stopwords
print(' '.join(stopwords.words('russian')[:20]))

и в во не что он на я с со как а то все она так его но да ты


Проблема: “To be or not to be"


## Нормализация
**Def.**  
Приведение токенов к единому виду для того, чтобы избавиться от поверхностной разницы в написании  

Подходы  
* сформулировать набор правил, по которым преобразуется токен  
Нью-Йорк → нью-йорк → ньюйорк → ньюиорк
* явно хранить связи между токенами (WordNet – Princeton)  
машина → автомобиль, Windows 6→ window

In [9]:
s = u'Нью-Йорк'
s1 = s.lower()
print(s1)

нью-йорк


In [20]:
import re
s2 = re.sub(r"\W", " ", s1, flags=re.U)
print(s2)

нью йорк


In [21]:
s3 = re.sub(r"й", u"и", s2, flags=re.U)
print(s3)

нью иорк


## Стемминг и Лемматизация
**Def.**  
Приведение грамматических форм слова и однокоренных слов к единой основе (lemma):
* Stemming – с помощью простых эвристических правил
  * Porter (Cambridge – 1980)
        5 этапов, на каждом применяется набор правил, таких как
            sses → ss (caresses → caress)
            ies → i (ponies → poni)

  * Lovins (1968)
  * Paice (1990)
  * другие
* Lemmatization – с использованием словарей и морфологического анализа преобразование к нормальной форме


## Стемминг

In [22]:
from nltk.stem.snowball import PorterStemmer
s = PorterStemmer()
print(s.stem('Tokenization'))
print( s.stem('stemming'))

from nltk.stem.snowball import RussianStemmer
r = RussianStemmer()
print (r.stem(u'Авиация'))
print (r.stem(u'национальный'))

token
stem
авиац
национальн


**Наблюдение**  
для сложных языков лучше подходит лемматизация

## Лемматизация

In [73]:
import pymorphy2

import pymorphy2
morph = pymorphy2.MorphAnalyzer()
for i in morph.parse(u'замок'):
    print(i)
    print(i.word, i.normal_form)

ModuleNotFoundError: No module named 'pymorphy2'

## Представление документов
**Boolean Model.** Присутствие или отсутствие слова в документе  
**Bag of Words.** Порядок токенов не важен - матрица док-ты vs токены, в ячейках - встречаемость токена в док-те 

*Погода была ужасная, принцесса была прекрасная.
Или все было наоборот?*

Координаты
* Мультиномиальные: количество токенов в документе
* Числовые: взвешенное количество токенов в документе

In [25]:
from sklearn.feature_extraction import DictVectorizer
# http://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.DictVectorizer.html

In [26]:
v = DictVectorizer(sparse=False)
D = [{'foo': 1, 'bar': 2}, {'foo': 3, 'baz': 1}]
v.fit(D)
X = v.transform(D)
X

array([[ 2.,  0.,  1.],
       [ 0.,  1.,  3.]])

In [27]:
v.inverse_transform(X)

[{'bar': 2.0, 'foo': 1.0}, {'baz': 1.0, 'foo': 3.0}]

In [28]:
v.transform({'foo': 4, 'unseen_feature': 3})

array([[ 0.,  0.,  4.]])

In [29]:
from collections import Counter
from nltk.tokenize import RegexpTokenizer

docs = [
    "Thank 40 you, Mr President.",
    "Madam President, I agree and recognise Turkey's European prospects, but if these prospects are to have an auspicious outcome, Turkey needs to:",
    "Madam President, firstly, I would like to express my sincerest thanks to the High Representative for including this important issue in the agenda at such an early stage.",
]

tokenizer = RegexpTokenizer('\w+|[^\w\s]+')
stopwords_eng = stopwords.words()

document_bags = list()

for d in docs:
    bag = Counter()
    text = d.lower()

    for t in tokenizer.tokenize(text):     
        if t in stopwords_eng:
            continue
            
        bag[t] += 1
    document_bags.append(bag)
    
document_bags

[Counter({',': 1, '.': 1, '40': 1, 'mr': 1, 'president': 1, 'thank': 1}),
 Counter({"'": 1,
          ',': 3,
          ':': 1,
          'agree': 1,
          'auspicious': 1,
          'european': 1,
          'madam': 1,
          'needs': 1,
          'outcome': 1,
          'president': 1,
          'prospects': 2,
          'recognise': 1,
          'turkey': 2}),
 Counter({',': 2,
          '.': 1,
          'agenda': 1,
          'early': 1,
          'express': 1,
          'firstly': 1,
          'high': 1,
          'important': 1,
          'including': 1,
          'issue': 1,
          'like': 1,
          'madam': 1,
          'president': 1,
          'representative': 1,
          'sincerest': 1,
          'stage': 1,
          'thanks': 1,
          'would': 1})]

In [30]:
v = DictVectorizer(sparse=False)
X = v.fit_transform(document_bags)
X.shape

(3, 31)

In [31]:
v.feature_names_

["'",
 ',',
 '.',
 '40',
 ':',
 'agenda',
 'agree',
 'auspicious',
 'early',
 'european',
 'express',
 'firstly',
 'high',
 'important',
 'including',
 'issue',
 'like',
 'madam',
 'mr',
 'needs',
 'outcome',
 'president',
 'prospects',
 'recognise',
 'representative',
 'sincerest',
 'stage',
 'thank',
 'thanks',
 'turkey',
 'would']

In [32]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer()
vectorizer.fit_transform(docs).todense()

matrix([[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1],
        [0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0,
         1, 0, 0, 1, 1, 1, 2, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 2, 2, 0, 0],
        [0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1,
         1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 2, 0, 1, 2, 0, 1, 0]], dtype=int64)

In [33]:
vectorizer.vocabulary_

{'40': 0,
 'agenda': 1,
 'agree': 2,
 'an': 3,
 'and': 4,
 'are': 5,
 'at': 6,
 'auspicious': 7,
 'but': 8,
 'early': 9,
 'european': 10,
 'express': 11,
 'firstly': 12,
 'for': 13,
 'have': 14,
 'high': 15,
 'if': 16,
 'important': 17,
 'in': 18,
 'including': 19,
 'issue': 20,
 'like': 21,
 'madam': 22,
 'mr': 23,
 'my': 24,
 'needs': 25,
 'outcome': 26,
 'president': 27,
 'prospects': 28,
 'recognise': 29,
 'representative': 30,
 'sincerest': 31,
 'stage': 32,
 'such': 33,
 'thank': 34,
 'thanks': 35,
 'the': 36,
 'these': 37,
 'this': 38,
 'to': 39,
 'turkey': 40,
 'would': 41,
 'you': 42}

## TF-IDF

Количество вхождений слова $t$ в документе $d$
$$
TF_{t,d} = term\!\!-\!\!frequency(t, d)
$$
Количество документов из $N$ возможных, где встречается $t$
$$
DF_t = document\!\!-\!\!fequency(t)
$$
$$
IDF_t = inverse\!\!-\!\!document\!\!-\!\!frequency(t) = \log \frac{N}{DF_t}
$$
TF-IDF
$$
TF\!\!-\!\!IDF_{t,d} = TF_{t,d} \times IDF_t
$$

Оценивает важность слова в контексте документа, являющегося частью корпуса
`

In [34]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer()
features = vectorizer.fit_transform(docs).todense()
features

matrix([[ 0.47952794,  0.        ,  0.        ,  0.        ,  0.        ,
          0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
          0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
          0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
          0.        ,  0.        ,  0.        ,  0.47952794,  0.        ,
          0.        ,  0.        ,  0.28321692,  0.        ,  0.        ,
          0.        ,  0.        ,  0.        ,  0.        ,  0.47952794,
          0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
          0.        ,  0.        ,  0.47952794],
        [ 0.        ,  0.        ,  0.20489728,  0.15582966,  0.20489728,
          0.20489728,  0.        ,  0.20489728,  0.20489728,  0.        ,
          0.20489728,  0.        ,  0.        ,  0.        ,  0.20489728,
          0.        ,  0.20489728,  0.        ,  0.        ,  0.        ,
          0.        ,  0.        ,  0.15582966,  0.        ,  0

In [35]:
vectorizer.vocabulary_

{'40': 0,
 'agenda': 1,
 'agree': 2,
 'an': 3,
 'and': 4,
 'are': 5,
 'at': 6,
 'auspicious': 7,
 'but': 8,
 'early': 9,
 'european': 10,
 'express': 11,
 'firstly': 12,
 'for': 13,
 'have': 14,
 'high': 15,
 'if': 16,
 'important': 17,
 'in': 18,
 'including': 19,
 'issue': 20,
 'like': 21,
 'madam': 22,
 'mr': 23,
 'my': 24,
 'needs': 25,
 'outcome': 26,
 'president': 27,
 'prospects': 28,
 'recognise': 29,
 'representative': 30,
 'sincerest': 31,
 'stage': 32,
 'such': 33,
 'thank': 34,
 'thanks': 35,
 'the': 36,
 'these': 37,
 'this': 38,
 'to': 39,
 'turkey': 40,
 'would': 41,
 'you': 42}

In [36]:
vectorizer.stop_words_

set()

In [37]:
docs

['Thank 40 you, Mr President.',
 "Madam President, I agree and recognise Turkey's European prospects, but if these prospects are to have an auspicious outcome, Turkey needs to:",
 'Madam President, firstly, I would like to express my sincerest thanks to the High Representative for including this important issue in the agenda at such an early stage.']

In [41]:
import numpy as np
# https://radimrehurek.com/gensim/
import gensim
from gensim.models import TfidfModel

vectorizer = CountVectorizer()
x = vectorizer.fit_transform(docs).todense()

corpus = [list(filter(lambda x: x[1] != 0, enumerate(np.asarray(row)[0]))) for row in x]
tfidf = TfidfModel(corpus)
print(tfidf[corpus[0]])

C:\Users\ACLion\Anaconda3\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


[(0, 0.5), (23, 0.5), (34, 0.5), (42, 0.5)]


## Байесовский классификатор

Дано

$\mathbf{x} \in X$ - описание документа $d$ из коллекции $D$  
$C_k \in C, \; k = 1,\ldots,K$ - целевая переменная

Теорема Байеса
$$
P(C_k \mid \mathbf{x}) = \frac{p(\mathbf{x} \mid C_k) p(C_k)}{p(\mathbf{x})} \propto p(\mathbf{x} \mid C_k) p(C_k)
$$

Принцип Maximum A-Posteriori
$$
C_{MAP} = \arg \max_k p(C_k | \mathbf{x})
$$

Байесовский классификатор — широкий класс алгоритмов классификации, основанный на принципе максимума апостериорной вероятности.  
Для классифицируемого объекта вычисляются функции правдоподобия каждого из классов, по ним вычисляются апостериорные вероятности классов.  
Объект относится к тому классу, для которого апостериорная вероятность максимальна.


## Naive Bayes

$x_j$ - слово на $j$-м месте в документе $\mathbf{x}$,  
$w^i \in V$ - слово из словаря $V$


Предположения
* conditional independence - слова внутри документа независимы
$$
p(x_i=w^s, x_j=w^r | C_k) = p(x_i=w^s | C_k) p(x_j=w^r | C_k)
$$
* postional independence - результат не зависит от позиции слова в документе
$$
P(x_i=w^s | C_k) = P(x_j=w^s | C_k) = P(x = w^s | C_k)
$$

Получаем
$$
p(\mathbf{x} | C_k) = p(x_1=w^{s_1}, \ldots, x_{|\mathbf{x}|}=w^{s_{|\mathbf{x}|}} | C_k) = \prod_{i=1}^{|\mathbf{x}|} p(x = w^{s_i} | C_k)
$$

**Почему NB хорошо работает?**  
Корректная оценка дает правильное предсказание, но правильное предсказание *не требует* корректной оценки


## Варианты NB

MAP
$$
C_{MAP} = \arg \max_k p(C_k) \prod_{i=1}^{|\mathbf{x}|} p(x = w^{s_i} | C_k)  = 
$$
$$
= \arg \max_k \left[ \log p(C_k) + \sum_{i=1}^{|\mathbf{x}|} \log p(x = w^{s_i} | C_k) \right]
$$
Априорные вероятности
$$
p(C_k) = N_{C_k}/{N}
$$
Likelihood $p(x = w^{s_i} | C_k)$
* BernoulliNB $p(x = w^{s_i} | C_k) = D_{w^{s_i}, C_k} / D_{C_k}$, $D$ - кол-во документов
* MultinomialNB $p(x = w^{s_i} | C_k) = T_{w^{s_i}, C_k} / T_{C_k}$, $T$ - кол-во токенов
* GaussianNB $p(x = w^{s_i} | C_k) = \mathcal{N}(\mu_k, \sigma_k^2)$, параметры из MLE


## Обучение NB

```
function nb_train(D,C):
	V = dictionary of tokens
	N = number of documents
	for Ck in C: # iterate over all classes
		N_Ck = number of documents in class Ck
		p(Ck) = N_Ck / N # Class prior
		D_Ck = Documents in class Ck		
		for w_i in V:			
			# multinomial, bernoulli, gaussian
			p(w_i|Ck) = count_likelihood(...)
	return V, p(Ck), p(w_i|Ck)
```

Алгоритмическая сложность: $O(|D| \langle |\mathbf{x}| \rangle + |C||V|)$

## Применение MultinomialNB


```
function nb_apply(d, C, V, p(Ck), p(w_i|Ck)):
	x = tokenize(d) # somehow	
	for Ck in C: # iterate over all classes
		score(Ck|x) = log p(Ck) # use class prior
		# use likelihoods
		for i in 1..|x|:		
			score(Ck|x) += log p(x_i|Ck)
	return arg max score(Ck|x)
```

Алгоритмическая сложность: $O(|C||\mathbf{x}|)$


## Сглаживание

Проблема: $p(свинки|мимими) = 0$

Решение:

$$ p(x=w_{s_i}|C_k) = \frac{ T_{w^{s_i}, C_k} + \alpha }{ T_{C_k} + \alpha|V|} $$


если $\alpha \geq 0$ - сглаживание Лапласа, если $0 \leq \alpha \leq 1$ - Лидстоуна



**+** (Удивительно) неплохо работает  
**+** Стабилен при смещении выборки  
**+** Оптимальный по производительности  

**-** Наивные предположения  
**-** Требует отбора признаков  

# SMS Spam  detection

https://www.kaggle.com/uciml/sms-spam-collection-dataset/data

In [42]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import accuracy_score

In [43]:
import pandas as pd
from sklearn.model_selection import train_test_split
import time

In [44]:
df = pd.read_csv('spam.csv', usecols=[0, 1], encoding='latin-1')
df.head()

,v1,v2
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [45]:
vectorizer = CountVectorizer()

y = pd.get_dummies(df['v1'])['spam']
X = vectorizer.fit_transform(df['v2'])

In [46]:
df['v1'].value_counts

<bound method IndexOpsMixin.value_counts of 0        ham
1        ham
2       spam
3        ham
4        ham
5       spam
6        ham
7        ham
8       spam
9       spam
10       ham
11      spam
12      spam
13       ham
14       ham
15      spam
16       ham
17       ham
18       ham
19      spam
20       ham
21       ham
22       ham
23       ham
24       ham
25       ham
26       ham
27       ham
28       ham
29       ham
        ... 
5542     ham
5543     ham
5544     ham
5545     ham
5546     ham
5547    spam
5548     ham
5549     ham
5550     ham
5551     ham
5552     ham
5553     ham
5554     ham
5555     ham
5556     ham
5557     ham
5558     ham
5559     ham
5560     ham
5561     ham
5562     ham
5563     ham
5564     ham
5565     ham
5566    spam
5567    spam
5568     ham
5569     ham
5570     ham
5571     ham
Name: v1, Length: 5572, dtype: object>

In [47]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)


In [48]:
def randomized_cv(model, param_grid, x_train, y_train):
    grid_search = RandomizedSearchCV(model, param_grid, cv=5, scoring='accuracy', n_iter=10)
    t_start = time.time()
    grid_search.fit(x_train, y_train)
    t_end = time.time()
    print('model {} best accuracy score is {}'.format(model.__class__.__name__, grid_search.best_score_))
    print('time for training is {} seconds'.format(t_end - t_start))
    return grid_search.best_estimator_

In [49]:
param_grid = {'alpha':[0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1, 1.5, 2, 5]}
model = MultinomialNB()
best_model = randomized_cv(model, param_grid, X_train, y_train)

model MultinomialNB best accuracy score is 0.981516206804179
time for training is 0.2098085880279541 seconds


In [50]:
y_pred = best_model.predict(X_test)
print(accuracy_score(y_test, y_pred))

0.980967917346


In [51]:
best_model.predict(X_test[:10])

array([0, 0, 1, 0, 1, 0, 0, 0, 0, 0], dtype=uint8)

In [52]:
vectorizer.inverse_transform(X_test[:10])

[array(['no', 'just', 'teaches', 'choose', 'funny', 'wife', 'tsunamis',
        'nobody', 'fact', 'hw', 'natural', 'happens', 'volcanoes', 'erupt',
        'arise', 'hurricanes', 'sway', 'aroundn', 'disasters'], 
       dtype='<U34'),
 array(['to', 'think', 'and', 'you', 'for', 'my', 'is', 'me', 'had', 'the',
        'on', 'cost', 'if', 'are', 'do', 'of', 'her', 'joke', 'one',
        'research', 'contact', 'school', 'sent', 'also', 'few', 'thinking',
        'less', 'schools', 'ones', 'scores', 'sophas', 'secondary',
        'application', 'applying', 'ogunrinde', 'expensive'], 
       dtype='<U34'),
 array(['to', 'you', 'call', '150p', 'pobox', 'that', 'we', 'know', 'out',
        'who', 'find', 'someone', 'fancies', '09058097218', 'ls15hb'], 
       dtype='<U34'),
 array(['only', 'in', 'ok', 'to', 'text', 'it', 'and', 'you', 'me', 'as',
        'your', 'the', 'soon', 'promise', 'if', 'can', 'getting', 'll',
        'let', 'know', 'out', 'morning', 'made'], 
       dtype='<U34'),
 ar

# Word2Vec

https://code.google.com/archive/p/word2vec/#Pre-trained_word_and_phrase_vectors

https://www.youtube.com/watch?v=EpJzLN8LL7Q&t=43s

In [53]:
from gensim.models import KeyedVectors
fn = "freebase-vectors-skipgram1000-en.bin.gz"
model = KeyedVectors.load_word2vec_format(fn)
model.most_similar('vacation')

FileNotFoundError: [Errno 2] No such file or directory: 'freebase-vectors-skipgram1000-en.bin.gz'

https://dumps.wikimedia.org/

In [ ]:
from gensim.corpora.wikicorpus import WikiCorpus
wiki = WikiCorpus('ruwiki-20171220-pages-articles-multistream.xml.bz2')

In [ ]:
from gensim.models.phrases import Phraser, Phrases
bigram = Phrases(wiki.get_texts())
bigram_transformer = Phraser(bigram)


def text_generator_bigram():
    for text in wiki.get_texts():
        yield bigram_transformer[[word.decode('utf8') for word in text]]
        
        
def text_generator_trigram():
    for text in wiki.get_texts():
        yield trigram_transformer[bigram_transformer[[word.decode('utf8') for word in text]]]

In [ ]:
from gensim.models.word2vec import Word2Vec
model = Word2Vec(size=100, window=7, min_count=10, workers=10)
model.build_vocab(text_generator_trigram())
model.train(text_generator_trigram())

fname = 'w2v_model_wiki'
model.save(fname)
model = Word2Vec.load(fname)
model.most_similar('токен')

Обработка текста для Java:  
https://stanfordnlp.github.io/CoreNLP/index.html  
https://opennlp.apache.org/docs/

# SpaCy



In [64]:
import spacy

# Load English tokenizer, tagger, parser, NER and word vectors
#python -m spacy download en
nlp = spacy.load('en')

# Process whole documents

text = '. '.join(df['v2'][:10])
doc = nlp(text)

# Find named entities, phrases and concepts
for entity in doc.ents:
    print(entity.text, entity.label_)


Free PERSON
2 CARDINAL
FA Cup EVENT
21st ORDINAL
May 2005 DATE
87121 DATE
rate)T&C ORG
08452810075over18 PERSON
Nah PERSON
3 week's DATE
Melle Melle PERSON
Oru Minnaminunginte Nurungu Vettam PERSON
9 CARDINAL
KL341 CARDINAL
Valid FAC
12 hours TIME
11 months DATE
U R ORG
Update GPE
The Mobile Update Co FREE ORG
08002986030 DATE


In [65]:
text

"Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat.... Ok lar... Joking wif u oni.... Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive entry question(std txt rate)T&C's apply 08452810075over18's. U dun say so early hor... U c already then say.... Nah I don't think he goes to usf, he lives around here though. FreeMsg Hey there darling it's been 3 week's now and no word back! I'd like some fun you up for it still? Tb ok! XxX std chgs to send, å£1.50 to rcv. Even my brother is not like to speak with me. They treat me like aids patent.. As per your request 'Melle Melle (Oru Minnaminunginte Nurungu Vettam)' has been set as your callertune for all Callers. Press *9 to copy your friends Callertune. WINNER!! As a valued network customer you have been selected to receivea å£900 prize reward! To claim call 09061701461. Claim code KL341. Valid 12 hours only.. Had your mobile 11 months or more

In [66]:
# Determine semantic similarities
doc1 = nlp(u'the fries were gross')
doc2 = nlp(u'worst potato ever')
doc1.similarity(doc2)


0.42931574228979374

In [67]:
# Determine semantic similarities
doc1 = nlp(u'men')
doc2 = nlp(u'women')
doc1.similarity(doc2)

0.86696618402938308